# Project No 1: Blog

We have come a long way so far and now we will walk through coding a django project from ground up. The project will be a blog that has some posts and shows the details of those posts once clicked.

Let's start simple. We make an app and call it main to act as a core for our project. Right after that, we'll make some views and urls.

In [1]:
from django.shortcuts import render

# Create your views here.

def index(request):
    pass

def posts(request):
    pass

def single_post(request):
    pass

`<slug>` is indeed a cool dynamic url tool which we'll talk about more deeply later on. Let's move on to urls. Now we did use `<slug:slug>` in our main app's urls, but the better practice would be to use it in a separate app called posts. Which is exactly how the project of this section is implemented.

In [ ]:
from django.urls import path
from . import views

urlpatterns = [
    path("", views.index),
    path("posts", views.posts),
    path("posts/<slug:slug>", views.single_post), # mywebsite.com/post/new-post
]

Great, now the addreses work. Time to add the master template. As we did before, make a new forlder at project root and call it `templates` inside which we will add a file `master.html` as below:

In [ ]:
<!-- master.html -->

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>
        {% block title %}
        Title
        {% endblock title %}
    </title>
    {% block header_refs %}
    {% endblock header_refs %}

</head>
<body>
    {% block content %}
        <h1>Page Content</h1>
    {% endblock content %}
</body>
</html>

Next, it's time to **extend** this master template in our `main` app. **Don't forget to add the master template directory to project settings!**

In [ ]:
<!-- Blog/main/templates/main/index.html -->
{% extends "master.html" %}

{% block title %}
    My Blog
{% endblock title %}

{% block content %}
    <h1>Home Page</h1>
{% endblock content %}

Render it in the view function and we're done with the base html files and pages.

In [ ]:
def index(request):
    render(request, "main/index.html")

### Project start checklist

This might be a good time to have a checklist on all the configurations needed when we start a new project.

1. start project using `django-admin`. 
2. start a new app (your main app) using `$ python manage.py startapp`
3. add the new app name to `INSTALLED_APPS`in project setitngs
4. make folders for master templates and master styles
5. add the needed blocks in `<body>` and `<head>` in master template
6. don't forget to add a `{% load static %}` on top of the master template file
7. add `BASE_DIR / 'templates'` to project settings at `TEMPLATES` part.
8. add `BASE_DIR / 'static/'` to project settings at `STATICFILES_DIRS = `
9. make you static and template folders in you app directory
10. write some sample HTML CSS
11. config some sample views and urls to test the app
12. Done!

### Addings Static Images

The process here is actually farely simple, say we need to treat image files like `png`, `jpg`, etc. just like a css file. Store them in a folder called `images` in static directory. Reference them in your HTML file like this: 

```HTML
<img src="{% static 'main/images/myimage.png' %}">
```

replage `main` with your app name and `myimage.png` with your image file name.

**Note: This approach only works for static images, the ones that will not change by user actoins, e. g. uploadin.**

### Clean code practices for posts

Each post has a code like this:

In [ ]:
<a href="{% url 'post_details' slug='postNumber' %}">
    <div class="post">
        <img src="{% static 'posts/includes/images/mountain.jpg' %}" alt="post_img">
        <div class="text_container">    
            <h5>Post Title</h5>
            <p>Post description, Lorem ipsum dolor, sit amet consectetur adipisicing elit. Molestias voluptatibus ex nisi numquam placeat odio voluptates assumenda voluptate est, iste iusto nostrum quod! Sit eius quos architecto quod optio laboriosam, sunt voluptas illum veritatis eaque.</p>
        </div>
    </div>
</a>

Of course repeating this in different places is not a good idea. Instead we'll use `includes` which we learned earlier.

Make a `includes` folder in the templates directory. Add a `post.html` file inside and copy and paste the code relate to each post in there. 

### Dynamic data for each post

It's time to start working with some database! He he he, not so fast! Our database here is a Python list in which there are dictionaries, each dictionary includes data for a single post: slug (id), title, author, image, content. There could be more but these are enough for now.

Add this list in your `views.py` file.

In [ ]:
# posts/views.py

all_posts = [
    {
        "slug": "1234",
        "title": "Learn Django",
        "author": "Erfan Rajati",
        "image": "mountain.jpg",
        "content": "Lorem ipsum dolor sit amet consectetur adipisicing elit. Aliquam consectetur quaerat, distinctio id facilis facere"
    },
    {
        "slug": "4311",
        "title": "Learn Python",
        "author": "Erfan Rajati",
        "image": "mountain.jpg",
        "content": "Lorem ipsum dolor sit amet consectetur adipisicing elit. Aliquam consectetur quaerat, distinctio id facilis facere"
    },
    {
        "slug": "1423",
        "title": "Learn Flask",
        "author": "Erfan Rajati",
        "image": "mountain.jpg",
        "content": "Lorem ipsum dolor sit amet consectetur adipisicing elit. Aliquam consectetur quaerat, distinctio id facilis facere"
    },
]

After you made that, the view function which is in charge of rendering the posts page should have a `context` parameter set to `all_posts` to send this data back to the HTML file.

In [ ]:
def posts(request):
    return render(request, "posts/index.html", context={"data":all_posts})

Same with the main HTML file that was rendering the posts page, now it will look like this:

In [ ]:
{% block content %}
    <h1>Posts Page</h1>
    <div class="post_list">
        {% for post in data %}
            {% include 'posts/includes/post.html' %}
        {% endfor %}
    </div>
{% endblock content %}

You'll notice that the `post.html` file does indeed have access to the `post` variable which the for loop is generating. So you can indeed use the data in that post inside the `post.html` file as well.

In [ ]:
{% load static %}
<a href="{% url 'post_details' slug='postNumber' %}">
    <div class="post">
        <img src="{% static 'posts/includes/images/mountain.jpg' %}" alt="post_img">
        <div class="text_container">    
            <h5>
                {{ post.title }}
            </h5>
            <p>
                {{ post.content }}
            </p>
        </div>
    </div>
</a>

Yeah I was fooled too! You can't just go `post["title"]` here, you must you a dot to access the keys of a given Python dictoinary in HTML file. 

Before I forger, a little bit of Django Template Language is needed in order to make post images dynamic too. But don't worry about it, it's all that's needed. For now :)

In [ ]:
<img src="{% static 'posts/includes/images/'|add:post.image %}" alt="post_img">